In [23]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, datasets
from tensorflow.keras.optimizers import SGD


In [ ]:
!unzip "/content/sample_data/spiral.zip" -d "/content/sample_data/"

In [15]:
# Set the path to your training and testing directories
train_dir = '/content/sample_data/spiral/train'
test_dir = '/content/sample_data/spiral/val'

# Define image dimensions and batch size
img_width, img_height = 229, 229  # You can adjust these dimensions based on your images
batch_size = 32

# Create data generators for training and testing sets
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=60,  # Random rotations up to 60 degrees
    width_shift_range=0.2,  # Random shifts up to 20% of the image width
    height_shift_range=0.2,  # Random shifts up to 20% of the image height
    zoom_range=0.2,   # Random zooms up to 20%
    horizontal_flip=True)   # Normalize pixel values between 0 and 1)

test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescaling for the testing set


In [16]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'  # Change to 'categorical' if you have more than two classes
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 198 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [17]:
def plot_images(images, labels):
    num_images = len(images)
    rows = int(np.sqrt(num_images))
    cols = int(np.ceil(num_images / rows))

    plt.figure(figsize=(10, 10))

    for i in range(num_images):
        plt.subplot(rows, cols, i+1)
        plt.imshow(images[i])
        plt.title(f"Label: {np.argmax(labels[i])}")
        plt.axis('off')

    plt.show()

In [ ]:
images, labels = train_generator.next()
plot_images(images, labels)


In [32]:
base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(229, 229, 3))

In [33]:
for layer in base_model.layers :
  layer.trainable=False

In [34]:
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

In [35]:
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.0001, momentum=0.9, decay=0.0005)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [36]:
batch_size = 32

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,# You can adjust the number of epochs
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,

)

Epoch 1/10
6/6 [==============================] - 35s 6s/step - loss: 0.8447 - accuracy: 0.5482 - val_loss: 0.7968 - val_accuracy: 0.4375
Epoch 2/10
6/6 [==============================] - 27s 4s/step - loss: 0.8155 - accuracy: 0.5783 - val_loss: 0.7104 - val_accuracy: 0.5312
Epoch 3/10
6/6 [==============================] - 26s 5s/step - loss: 0.9475 - accuracy: 0.4699 - val_loss: 0.6671 - val_accuracy: 0.6250
Epoch 4/10
6/6 [==============================] - 28s 4s/step - loss: 0.7226 - accuracy: 0.6446 - val_loss: 0.6346 - val_accuracy: 0.6250
Epoch 5/10
6/6 [==============================] - 26s 4s/step - loss: 0.7826 - accuracy: 0.5783 - val_loss: 0.5689 - val_accuracy: 0.6562
Epoch 6/10
6/6 [==============================] - 28s 5s/step - loss: 0.6757 - accuracy: 0.6386 - val_loss: 0.5501 - val_accuracy: 0.6875
Epoch 7/10
6/6 [==============================] - 27s 4s/step - loss: 0.7101 - accuracy: 0.6145 - val_loss: 0.5167 - val_accuracy: 0.8125
Epoch 8/10
6/6 [==================

In [38]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

2/2 [==============================] - 7s 2s/step - loss: 0.4872 - accuracy: 0.7200
Test Loss: 0.4871828556060791, Test Accuracy: 0.7200000286102295


In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 22327842 (85.17 MB)
Trainable params: 22293410 (85.04 MB)
Non-trainable params: 34432 (134.50 KB)
__________